In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as sc
# from scipy.stats import linregress
from pprint import pprint

# Import the OpenWeatherMap API key
from api_keys import api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# Load your CSV file
df = pd.read_csv('Tourist_Travel_Europe.csv')
df.head()

,Tourist_ID,Country_Visited,City_Visited,Mode_of_Travel,Travel_Duration_Days,Number_of_Companions,Total_Travel_Cost,Accommodation_Type,Main_Purpose,Season_of_Visit
0,1,Germany,Hamburg,Flight,8,4,1060,Hotel,Family Visit,Summer
1,2,Germany,Berlin,Car,16,3,3972,Hostel,Family Visit,Winter
2,3,UK,Manchester,Bicycle,8,2,666,Camping,Business,Winter
3,4,Greece,Thessaloniki,Bicycle,12,3,3644,Hotel,Business,Summer
4,5,Greece,Thessaloniki,Flight,5,3,1885,Airbnb,Leisure,Spring


In [3]:
df['lat'] = ""
df['long'] = ""
df.head()

,Tourist_ID,Country_Visited,City_Visited,Mode_of_Travel,Travel_Duration_Days,Number_of_Companions,Total_Travel_Cost,Accommodation_Type,Main_Purpose,Season_of_Visit,lat,long
0,1,Germany,Hamburg,Flight,8,4,1060,Hotel,Family Visit,Summer,,
1,2,Germany,Berlin,Car,16,3,3972,Hostel,Family Visit,Winter,,
2,3,UK,Manchester,Bicycle,8,2,666,Camping,Business,Winter,,
3,4,Greece,Thessaloniki,Bicycle,12,3,3644,Hotel,Business,Summer,,
4,5,Greece,Thessaloniki,Flight,5,3,1885,Airbnb,Leisure,Spring,,


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Tourist_ID            1000 non-null   int64 
 1   Country_Visited       1000 non-null   object
 2   City_Visited          1000 non-null   object
 3   Mode_of_Travel        1000 non-null   object
 4   Travel_Duration_Days  1000 non-null   int64 
 5   Number_of_Companions  1000 non-null   int64 
 6   Total_Travel_Cost     1000 non-null   int64 
 7   Accommodation_Type    1000 non-null   object
 8   Main_Purpose          1000 non-null   object
 9   Season_of_Visit       1000 non-null   object
 10  lat                   1000 non-null   object
 11  long                  1000 non-null   object
dtypes: int64(4), object(8)
memory usage: 93.9+ KB


In [ ]:
#TEST
city = "Thessaloniki, Greece"
#Geocode to get location
params = {
    "text": city,
    "format": "json",
    "apiKey": api_key
}
# Set base URL
base_url = "https://api.geoapify.com/v1/geocode/search"
response = requests.get(base_url, params=params)
print(response.status_code)
# Print the json (pretty printed)
data = response.json()
pprint(data)

In [ ]:
for index, row in df.iterrows():

    # Note that we used "" to specify initial entry.# TEST
    city = row["City_Visited"]
    country = row["Country_Visited"]
    # GEOCODE to get Lat/Long
    # Build URL using the geocode endpoint
    params = {
        "text": city,
        "country": country,
        "format": "json",
        "apiKey": api_key
    }
    # Set base URL
    base_url = "https://api.geoapify.com/v1/geocode/search"
    # Run request
    response = requests.get(base_url, params=params)
    print(response.status_code)
    data = response.json()
    # Extract lat/lon
    results = data.get("results", [])
    # Resiliency/Error Handling
    if len(results) > 0:
        location = results[0]
        latitude = location.get("lat")
        longitude = location.get("lon")
        address = location.get("formatted")
        df.at[index, "lat"] = latitude
        df.at[index, "long"] = longitude
        # Print results
        print(f"{address} is located at {latitude}, {longitude}")
    else:
        print("OH NOOOOO!! Your response did not include any results :'(")

In [ ]:
df.head()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Tourist_ID            1000 non-null   int64 
 1   Country_Visited       1000 non-null   object
 2   City_Visited          1000 non-null   object
 3   Mode_of_Travel        1000 non-null   object
 4   Travel_Duration_Days  1000 non-null   int64 
 5   Number_of_Companions  1000 non-null   int64 
 6   Total_Travel_Cost     1000 non-null   int64 
 7   Accommodation_Type    1000 non-null   object
 8   Main_Purpose          1000 non-null   object
 9   Season_of_Visit       1000 non-null   object
 10  lat                   1000 non-null   object
 11  long                  1000 non-null   object
dtypes: int64(4), object(8)
memory usage: 93.9+ KB


In [ ]:
df.to_csv("./Tourist_Travel_With_Coordinates.csv")

In [ ]:
df = pd.read_csv('Tourist_Travel_With_Coordinates.csv')

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine("sqlite:///touristtravel.sqlite")

In [ ]:
df.to_sql(name="touristtravel", con=engine, index=False, if_exists="append", method="multi")